In [1]:
import pandas as pd
import os

In [2]:
model = "glm_eccentricity"

In [3]:
sub_list = ["sub-01","sub-02","sub-03",
            "sub-04","sub-05","sub-06",
            "sub-07","sub-08","sub-09",
            "sub-10","sub-11","sub-12",
            "sub-13","sub-15","sub-16",
            "sub-17","sub-18","sub-19",
            "sub-21","sub-22","sub-23",
            "sub-24","sub-25","sub-26",
            "sub-27","sub-28","sub-30",
            "sub-31","sub-32","sub-33",]

run_list = ["run-01","run-02","run-03","run-04"]

In [4]:
base_folder = '/home/lisz/Desktop/Time_project/time_fmri'

In [5]:
def open_file():
    fldr = base_folder+'/behavioral_data/'+sub+'/'
    pth = fldr+sub+'_'+run+'.csv'
    df = pd.read_csv(pth, usecols=["fmri_resp.started",
                           "fmri_resp.rt",
                           "ITI.started",
                           "standard.started", 
                           "isi_pause.started",
                           "key_resp.started",
                           "key_resp_rt",
                           "vis_area",
                           "exclusion"])
    # check when the fmri machine was turned on
    exp_beg = df['fmri_resp.started'][0] + df['fmri_resp.rt'][0]
    #print("start of the fmri machine recording", exp_beg)

    # normalize the time of interest to the fmri machine timing
    df['pd_start'] = df['isi_pause.started'].sub(exp_beg)
    df['response_start'] = df['key_resp.started'].sub(exp_beg)
    df['standard_start'] = df['standard.started'].sub(exp_beg)
    df['ITI_start'] = df['ITI.started'].sub(exp_beg)
    
    df['ITI_end'] = df['standard_start']

    df['response_end'] = df['response_start']+df['key_resp_rt']

    # set the beginnings of the events
    df['standard_end'] = df['standard_start']+2

    df['pd_tranzit_start'] = df['pd_start']
    df['pd_tranzit_end'] = df['pd_start']+3

    df['pd_plateau_start'] = df['pd_start']+3
    df['pd_plateau_end'] = df['response_start']
    df['rt100'] = 0.1
    df['rt100_end'] = df['response_end']+df['rt100']
    return df

In [6]:
def create_df(df, clmn, var):
    # (df['column1'] == var1) & (df['column2'] == var2)
    # (df['column1'] == var1)
    periph = pd.DataFrame(df.loc[var])
    for index, row in periph.iterrows():
        if row['exclusion'] == 'exclude':
            periph.at[index, clmn] = 0
        else:
            periph.at[index, clmn] = 1
    # set vis_area to one since it is a condition
    return periph

In [7]:
def create_ev_with_failed_trials(df, clmn, var):
    # (df['column1'] == var1) & (df['column2'] == var2)
    # (df['column1'] == var1)
    periph = pd.DataFrame(df.loc[var])
    for index, row in periph.iterrows():
        if row['exclusion'] == 'exclude':
            periph.at[index, clmn] = 1
        else:
            periph.at[index, clmn] = 0
    # set vis_area to one since it is a condition
    
    return periph

In [8]:
""" this function adds a new row into exsisting dataframe,
where the "end" value from the row above placed in the first column,
"beg" value from the row below placed in the second column and
new row's condition should be equal to 0"""

def add_rows(subset, begins, ends, variabl):
    # Create a new empty DataFrame with the same columns as the original
    new_df = pd.DataFrame(columns=subset.columns)

    # Loop through the rows of the original DataFrame
    for i in range(subset.shape[0]):
        # Get the values of the current row
        beg = subset.iloc[i][begins]
        end = subset.iloc[i][ends]
        condition = subset.iloc[i][variabl]
        
        # Append the current row to the new DataFrame,
        new_df = new_df.append({'beg': beg, 'end': end, 'condition': condition}, ignore_index=True)

        # If this is not the last row, add a new row with the \"end\" value from the current row,
        # the \"beg\" value from the next row, and a condition of 0\n",
        if i < subset.shape[0] - 1:
            beg_next = subset.iloc[i+1][begins]
            new_df = new_df.append({'beg': end, 'end': beg_next, 'condition': 0}, ignore_index=True)
    txt_df = new_df[['beg','end','condition']].copy()
    txt_df = txt_df.reset_index(drop=True)
    txt_df = txt_df.dropna()
    # Return the new DataFrame
    return txt_df

In [9]:
def save_txt(df,contrast):
    fldr_save = base_folder+'/data/derivatives/'+model+'/EVs'
    file_dir = os.path.join(fldr_save, sub, contrast)
    os.makedirs(file_dir, exist_ok=True)
    file_path = os.path.join(file_dir, f"{sub}_{run}_{contrast}.txt")
    df.to_csv(file_path, sep='\t', header=False, index=False)

In [10]:
def merge_sequential_zeros(df):
    merged_df = pd.DataFrame(columns=df.columns)
    start = None
    end = None
    condition = None

    for index, row in df.iterrows():
        if row["condition"] == 0.0:
            if start is None:
                start = row["beg"]
                condition = 0.0
            end = row["end"]
        else:
            if start is not None:
                merged_df = merged_df.append({"beg": start, "end": end, "condition": condition}, ignore_index=True)
                start = None
                end = None
                condition = None
            merged_df = merged_df.append(row, ignore_index=True)

    if start is not None:
        merged_df = merged_df.append({"beg": start, "end": end, "condition": condition}, ignore_index=True)

    return merged_df

In [11]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_ev_with_failed_trials(a, 'vis_area', (a['vis_area'] != None))
        # since the exclusion criteria is a behavioral resonse,
        c = add_rows(b, 'standard_start', 'response_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'failed')

In [13]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] != None))
        c = add_rows(b, 'pd_plateau_start', 'pd_plateau_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_plateau')

In [ ]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'key_resp_rt', (a['key_resp_rt'] != None))
        c = add_rows(b, 'response_start', 'response_end', 'key_resp_rt')
        d = merge_sequential_zeros(c)
        save_txt(d,'visual_stim')

In [111]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'key_resp_rt', (a['key_resp_rt'] != None))
        c = add_rows(b, 'response_end', 'rt100_end', 'key_resp_rt')
        d = merge_sequential_zeros(c)
        save_txt(d,'rt100')

In [14]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'key_resp_rt', (a['key_resp_rt'] != None))
        c = add_rows(b, 'response_start', 'response_end', 'key_resp_rt')
        d = merge_sequential_zeros(c)
        save_txt(d,'response')

In [15]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] != None))
        c = add_rows(b, 'standard_start', 'standard_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'standard')

In [16]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] != None))
        c = add_rows(b, 'pd_tranzit_start', 'pd_tranzit_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_tranzit')

In [12]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'perifoveal'))
        c = add_rows(b, 'standard_start', 'standard_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'standard_perifoveal')

In [13]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'peripheral'))
        c = add_rows(b, 'standard_start', 'standard_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'standard_peripheral')

In [43]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'peripheral'))
        c = add_rows(b, 'response_start', 'response_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'response_peripheral')

In [44]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'perifoveal'))
        c = add_rows(b, 'response_start', 'response_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'response_perifoveal')

In [45]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] != None))
        c = add_rows(b, 'pd_tranzit_start', 'pd_plateau_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD')

In [46]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'perifoveal'))
        c = add_rows(b, 'pd_tranzit_start', 'pd_plateau_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_foveal')

In [47]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'peripheral'))
        c = add_rows(b, 'pd_tranzit_start', 'pd_plateau_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_periph')

In [50]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'perifoveal'))
        c = add_rows(b, 'pd_tranzit_start', 'pd_tranzit_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_tranzit_foveal')

In [51]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'peripheral'))
        c = add_rows(b, 'pd_tranzit_start', 'pd_tranzit_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_tranzit_periph')

In [52]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'perifoveal'))
        c = add_rows(b, 'pd_plateau_start', 'pd_plateau_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_plateau_foveal')

In [53]:
for sub in sub_list:
    for run in run_list:
        a = open_file()
        b = create_df(a, 'vis_area', (a['vis_area'] == 'peripheral'))
        c = add_rows(b, 'pd_plateau_start', 'pd_plateau_end', 'vis_area')
        d = merge_sequential_zeros(c)
        save_txt(d,'PD_plateau_periph')